<a href="https://colab.research.google.com/github/ferdinandrafols/IA_LLMs/blob/main/DiscursosObamaScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import re
import json
from urllib.parse import urljoin

BASE_LIST_URL = "https://obamawhitehouse.archives.gov/briefing-room/speeches-and-remarks"
BASE_DOMAIN = "https://obamawhitehouse.archives.gov"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; ObamaCorpusBot/1.0; +https://example.com/bot)"
}

def get_soup(url):
    resp = requests.get(url, headers=HEADERS, timeout=20)
    resp.raise_for_status()
    return BeautifulSoup(resp.text, "html.parser")

def extract_speech_links_from_page(list_url):
    """
    Dada uma página da lista de discursos,
    retorna uma lista de URLs completas para cada discurso
    + o próprio soup para paginação.
    """
    soup = get_soup(list_url)
    links = []

    # Os discursos geralmente aparecem em .view-content .views-row
    for row in soup.select(".view-content .views-row"):
        a = row.find("a", href=True)
        if not a:
            continue
        href = a["href"]
        full_url = urljoin(BASE_DOMAIN, href)
        links.append(full_url)

    return links, soup

def find_next_page(soup):
    """
    Tenta encontrar link de próxima página na paginação.
    Isso pode precisar de ajuste se o HTML mudar.
    """
    # Padrão comum: link com título "Go to next page"
    next_link = soup.find("a", title="Go to next page")
    if next_link and next_link.get("href"):
        return urljoin(BASE_DOMAIN, next_link["href"])

    # fallback: procurar algo com 'next' na paginação
    pager_next = soup.select_one(".pager-next a, li.next a")
    if pager_next and pager_next.get("href"):
        return urljoin(BASE_DOMAIN, pager_next["href"])

    return None

def extract_title(soup):
    """
    Tenta extrair o título do discurso.
    Usa vários seletores comuns como fallback.
    """
    # Padrões comuns do site
    title = None
    h1 = soup.find("h1", class_="page-title") or \
         soup.find("h1", id="page-title") or \
         soup.find("h1")
    if h1 and h1.get_text(strip=True):
        title = h1.get_text(strip=True)
    else:
        # fallback: <title> da página
        if soup.title and soup.title.get_text(strip=True):
            title = soup.title.get_text(strip=True)

    return title or ""

def extract_date(soup):
    """
    Tenta extrair a data do discurso.
    Isso pode variar de acordo com o template.
    """
    # Caso 1: span com classe de data
    date_span = soup.find("span", class_="date-display-single")
    if date_span and date_span.get_text(strip=True):
        return date_span.get_text(strip=True)

    # Caso 2: time tag
    time_tag = soup.find("time")
    if time_tag and time_tag.get_text(strip=True):
        return time_tag.get_text(strip=True)

    # Caso 3: div/linha de metadados
    meta_div = soup.find(class_="submitted") or soup.find("p", class_="datestamp")
    if meta_div and meta_div.get_text(strip=True):
        txt = meta_div.get_text(" ", strip=True)
        return txt

    # Se nada der certo, vazio (ainda assim o JSON fica usável)
    return ""

def extract_main_text_from_speech(soup):
    """
    Extrai o texto principal do discurso da página HTML.
    Pode ser necessário ajustar se o HTML do site mudar.
    """
    # Muitas páginas usam div.field-item para o conteúdo do discurso
    content_divs = soup.select(".field-item.even, .field-item.odd")
    if not content_divs:
        # fallback: tentar <article>
        article = soup.find("article")
        if article:
            text = article.get_text(separator="\n")
        else:
            # fallback final: texto da página toda
            text = soup.get_text(separator="\n")
    else:
        text_parts = [div.get_text(separator="\n") for div in content_divs]
        text = "\n".join(text_parts)

    # Limpeza básica
    text = re.sub(r"\r", "\n", text)
    text = re.sub(r"\n{2,}", "\n", text)
    text = text.strip()

    return text

def extract_speech_data(url):
    """
    Dado a URL de um discurso, retorna um dict:
    {
        "title": ...,
        "date": ...,
        "url": ...,
        "text": ...
    }
    """
    soup = get_soup(url)
    title = extract_title(soup)
    date = extract_date(soup)
    text = extract_main_text_from_speech(soup)

    return {
        "title": title,
        "date": date,
        "url": url,
        "text": text
    }

def crawl_all_obama_speeches(max_pages=None):
    """
    Percorre todas as páginas de listagem de discursos.
    max_pages: se for None, vai até acabar.
    """
    all_links = []
    page_url = BASE_LIST_URL
    page_count = 0

    while page_url:
        page_count += 1
        print(f"[INFO] Coletando página {page_count}: {page_url}")

        links, soup = extract_speech_links_from_page(page_url)
        print(f"  -> {len(links)} discursos encontrados nesta página.")
        all_links.extend(links)

        if max_pages is not None and page_count >= max_pages:
            break

        next_url = find_next_page(soup)
        if not next_url:
            print("[INFO] Não há próxima página. Fim da paginação.")
            break

        page_url = next_url
        time.sleep(1.0)  # educado com o servidor

    # Remove duplicados preservando a ordem
    seen = set()
    unique_links = []
    for url in all_links:
        if url not in seen:
            seen.add(url)
            unique_links.append(url)

    print(f"[INFO] Total de discursos únicos: {len(unique_links)}")
    return unique_links

def build_obama_jsonl_corpus(output_path="obama_speeches.jsonl",
                             log_every=20,
                             max_pages=None):
    """
    Gera um arquivo JSONL onde cada linha é:
    {"title": ..., "date": ..., "url": ..., "text": ...}
    """
    speech_links = crawl_all_obama_speeches(max_pages=max_pages)

    total_docs = 0
    with open(output_path, "w", encoding="utf-8") as f_out:
        for i, url in enumerate(speech_links, start=1):
            try:
                print(f"[INFO] ({i}/{len(speech_links)}) Processando discurso: {url}")
                data = extract_speech_data(url)

                # Se quiser descartar páginas vazias:
                if not data["text"]:
                    print("  -> texto vazio, pulando.")
                    continue

                line = json.dumps(data, ensure_ascii=False)
                f_out.write(line + "\n")
                total_docs += 1

                if i % log_every == 0:
                    print(f"  -> discursos acumulados escritos: {total_docs}")

                time.sleep(0.5)  # evita sobrecarga no servidor
            except Exception as e:
                print(f"[ERRO] Falha ao processar {url}: {e}")
                continue

    print(f"[FIM] Corpus JSONL gerado em '{output_path}' com {total_docs} discursos.")

if __name__ == "__main__":
    # Para testar com poucas páginas, use por exemplo:
    # build_obama_jsonl_corpus(max_pages=2)
    build_obama_jsonl_corpus()


[INFO] Coletando página 1: https://obamawhitehouse.archives.gov/briefing-room/speeches-and-remarks
  -> 10 discursos encontrados nesta página.
[INFO] Coletando página 2: https://obamawhitehouse.archives.gov/briefing-room/speeches-and-remarks?term_node_tid_depth=31&page=1
  -> 10 discursos encontrados nesta página.
[INFO] Coletando página 3: https://obamawhitehouse.archives.gov/briefing-room/speeches-and-remarks?term_node_tid_depth=31&page=2
  -> 10 discursos encontrados nesta página.
[INFO] Coletando página 4: https://obamawhitehouse.archives.gov/briefing-room/speeches-and-remarks?term_node_tid_depth=31&page=3
  -> 10 discursos encontrados nesta página.
[INFO] Coletando página 5: https://obamawhitehouse.archives.gov/briefing-room/speeches-and-remarks?term_node_tid_depth=31&page=4
  -> 10 discursos encontrados nesta página.
[INFO] Coletando página 6: https://obamawhitehouse.archives.gov/briefing-room/speeches-and-remarks?term_node_tid_depth=31&page=5
  -> 10 discursos encontrados nesta 